In [2]:
import tensorflow as tf
import numpy as np

Т.к. мы не планируем обучать какую-либо модель, определим заглушки заданного размера

In [3]:
x_inp = tf.placeholder(tf.float32, [5, 5])
w_inp = tf.placeholder(tf.float32, [3, 3])

В tensorflow двумерные свертки реализуются  с помощью функции conv2d, которая принимает на вход тензор сверточных весов и тензор карт признаков набора изображений
[размер_батча, высота, ширины, каналы]. 
<br>
Например, если мы используем мини-батчи размером 32 изображения в каждом и обучаем RGB-изображения лиц размером 28х28, то итоговая размерность тензора будет: [32, 28, 28, 3], значит каждый минибатч подающийся на вход сети будет содержать около 75 тысяч чисел
<br>
Размерность тензора сверточных весов определяется размерами ядра свертки и числом каналов как на входе так и на выходе: [высота, ширина, входные каналы, выходные каналы]. Так для фильтра 3х3 применяемый к rgb изображению и дающему на выходе 32 карты признаков мы получим размерности: [3, 3, 3, 32] и всего в нем будет 288 весов и на выходе они превратят исходную картинку в картинку 26х26х32 = 21632 числа, что в 50 тысяч раз меньше аналогичной полносвязной сети (17 миллионов чисел)

In [4]:
x = tf.reshape(x_inp, [1, 5, 5, 1])
w = tf.reshape(w_inp, [3, 3, 1, 1])

In [5]:
x_valid = tf.nn.conv2d(x, w, strides=[1, 1, 1, 1], padding='VALID')
x_same = tf.nn.conv2d(x, w, strides=[1, 1, 1, 1], padding='SAME')
x_valid_half = tf.nn.conv2d(x, w, strides=[1, 2, 2, 1], padding='VALID')
x_same_half = tf.nn.conv2d(x, w, strides=[1, 2, 2, 1], padding='SAME')

In [6]:
x = ([[0, 1, 2, 1 ,0],
         [4, 1, 0, 1, 0],
         [2, 0, 1, 1, 1],
         [1, 2, 3, 1, 0],
         [0, 4, 3, 2, 0]])

w = ([[0, 1, 0],
         [1, 0, 1],
         [2, 1, 0]])

In [7]:
sess = tf.Session()
y_valid, y_same, y_valid_half, y_same_half = sess.run([x_valid, x_same, x_valid_half, x_same_half], feed_dict={x_inp: x, w_inp: w})

In [10]:
print("padding=VALID:\n", y_valid[0, :, :, 0])
print("padding=SAME:\n", y_same[0, :, :, 0])
print("padding=VALID, stride 2:\n", y_valid_half[0, :, :, 0])
print("padding=SAME, stride 2:\n", y_same_half[0, :, :, 0])


padding=VALID:
 [[ 9.  5.  4.]
 [ 8.  8. 10.]
 [ 8. 15. 12.]]
padding=SAME:
 [[ 5. 11.  4.  3.  3.]
 [ 3.  9.  5.  4.  4.]
 [ 5.  8.  8. 10.  3.]
 [ 4.  8. 15. 12.  6.]
 [ 5.  5.  9.  4.  2.]]
padding=VALID, stride 2:
 [[ 9.  4.]
 [ 8. 12.]]
padding=SAME, stride 2:
 [[5. 4. 3.]
 [5. 8. 3.]
 [5. 9. 2.]]


Субдескритизация в tensorflow.
Создадим заглушку и приведем её к нужной размерности

In [11]:
x_inp = tf.placeholder(tf.float32, [4, 4])
x = tf.reshape(x_inp, [1, 4, 4, 1])

Теперь можем определять операции субдискретизации с помощью специальной функции (попробуем шаг 1 и 2)

In [12]:
x_valid = tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 1, 1, 1], padding='VALID')
x_valid_half = tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')

In [13]:
x = np.array([[0, 1, 2, 1],
                        [4, 1, 0, 1],
                        [2, 0, 1, 1],
                        [1, 2, 3, 1]])
y_valid, y_valid_half = sess.run([x_valid, x_valid_half], feed_dict={x_inp: x})

In [14]:
print( "padding=VALID:\n", y_valid[0, :, :, 0])
print( "padding=VALID, stride 2:\n", y_valid_half[0, :, :, 0])

padding=VALID:
 [[4. 2. 2.]
 [4. 1. 1.]
 [2. 3. 3.]]
padding=VALID, stride 2:
 [[4. 2.]
 [2. 3.]]
